# Hetero-NN Customize Models & Loss

We can customize the top model and the bottom model in the Hetero NN.
The model_zoo module was introduced in FATE 1.10, located under federatedml.nn.model_zoo. This module allows you to customize your own PyTorch model, provided that it is developed based on torch.nn.Module and implements the forward interface. For more information, see the PyTorch documentation  [Pytorch Module](https://pytorch.org/docs/stable/notes/modules.html#a-simple-custom-module) on custom modules. To use your custom model in a federated task, simply place it in the federatedml/nn/model_zoo directory and specify the module and model class through the interface when submitting the task. Hetero-NN components will automatically search and import the model you have implemented.

You can also define your own loss class in a similar way. The you can place your loss class under the loss module, located under federatedml.nn.loss.
We recommend you read these two tutorials before reading this tutorial: [Customize loss function](Homo-NN-Customize-Loss.ipynb), [Customize Model](Homo-NN-Customize-Model.ipynb)

As an example, we consider reuse the task of MNIST handwriting recognition of last hetero-nn tutorial.

## Prepare MNIST Data

Please download the guest/host MNIST dataset from the link below and place it in the project examples/data folder:

- guest data: https://webank-ai-1251170195.cos.ap-guangzhou.myqcloud.com/fate/examples/data/mnist_guest.zip

- host data: https://webank-ai-1251170195.cos.ap-guangzhou.myqcloud.com/fate/examples/data/mnist_host.zip
  
The mnist_guest is a simplified version of the MNIST dataset, with a total of ten categories, which are classified into 0-9 10 folders according to labels. The mnist_host has the same images as the mnist_guest, but it is not labeled.

## Dataset

In version FATE-1.10, FATE introduces a new base class for datasets called Dataset, which is based on PyTorch's Dataset class. This class allows users to create custom datasets according to their specific needs. The usage is similar to that of PyTorch's Dataset class, with the added requirement of implementing two additional interfaces when using FATE-NN for data reading and training: load() and get_sample_ids().

To create a custom dataset in Hetero-NN, users need to:

- Develop a new dataset class that inherits from the Dataset class
- Implement the \_\_len\_\_() and \_\_getitem\_\_() methods, which are consistent with PyTorch's Dataset usage. The \_\_len\_\_() method should return the length of the dataset, while the \_\_getitem\_\_() method should return the corresponding data at the specified index. **However, please notice that different \_\_getitem\_\_() methods may have different behaviors between different parties. In the guest party(party with labels), _\_getitem\_\_() method return features and labels, while in the host parties(parties without label), _\_getitem\_\_() method return features only.** 
- Implement the load(), get_sample_ids(), get_classes() methods
  
For those unfamiliar with PyTorch's Dataset class, more information can be found in the PyTorch documentation: [Pytorch Dataset Documentation](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)

## Customize Bottom/Top Model

Name the model code bottom_net.py,  you can put it directly under federatedml/nn/model_zoo or use the shortcut interface of jupyter notebook: save_to_fate, to save it directly to federatedml/nn/model_zoo. This is the bottom model structure we define for feature extraction.

In [1]:
from pipeline.component.nn import save_to_fate

/data/projects/fate/env/python/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%%save_to_fate model bottom_net.py
import torch as t
from torch import nn
from torch.nn import Module
import logging
import numpy as np

class BottomNet(nn.Module):

    def __init__(self):
        super(BottomNet, self).__init__()
        self.cuda = True
        self.seq = t.nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5),
            nn.MaxPool2d(kernel_size=3),
            nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3),
            nn.AvgPool2d(kernel_size=3)
        )

        self.fc = t.nn.Sequential(   # extracted feature is a 8-dim embedding
            nn.Linear(48, 32),
            nn.ReLU(),
            nn.Linear(32, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = x[0]
        x = self.seq(x)
        x = x.flatten(start_dim=1)
        x = self.fc(x)
        return x

And this is the top model we define for classification, we named it as top_model.py.

In [3]:
%%save_to_fate model dummy.py
import torch as t
from torch import nn
from torch.nn import Module

class DummyNet(nn.Module):

    def __init__(self):
        super(DummyNet, self).__init__()
        self.fc = t.nn.Linear(2,1)

    def forward(self, x):
        x = self.fc(x)
        return x

In [4]:
%%save_to_fate model top_net.py
import torch as t
from torch import nn
from torch.nn import Module

class TopNet(nn.Module):

    def __init__(self):
        super(TopNet, self).__init__()
        self.fc = t.nn.Sequential(   
            nn.Linear(10, 10),
            t.nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.fc(x)
        return x

## Use Cust Loss

Using Cust Loss is exactly the same as Homo-NN, see: [Customize loss function](Homo-NN-Customize-Loss.ipynb). Here we use a new CrossEntropyLoss.

In [5]:
%%save_to_fate loss ce.py
import torch as t
from federatedml.util import consts
from torch.nn.functional import one_hot
from fate_arch.common import log

LOGGER = log.getLogger()

def cross_entropy(p2, p1, reduction='mean'):
    p2 = p2 + consts.FLOAT_ZERO  # to avoid nan
    assert p2.shape == p1.shape
    if reduction == 'sum':
        return -t.sum(p1 * t.log(p2))
    elif reduction == 'mean':
        return -t.mean(t.sum(p1 * t.log(p2), dim=1))
    elif reduction == 'none':
        return -t.sum(p1 * t.log(p2), dim=1)
    else:
        raise ValueError('unknown reduction')


class CrossEntropyLoss(t.nn.Module):

    """
    A CrossEntropy Loss that will not compute Softmax
    """

    def __init__(self, reduction='mean'):
        super(CrossEntropyLoss, self).__init__()
        self.reduction = reduction

    def forward(self, pred, label):
        LOGGER.info(label.shape)
        LOGGER.info(label[0])
        one_hot_label = one_hot(label.to(t.int64))
        one_hot_label = t.flatten(one_hot_label, start_dim=1)
        LOGGER.info(one_hot_label.shape)
        LOGGER.info(one_hot_label[0])
        LOGGER.info("pred")
        LOGGER.info(pred.shape)
        LOGGER.info(pred[0])
        
        loss_ = cross_entropy(pred, one_hot_label, self.reduction)

        return loss_

Then, we can use our models & loss in the Hetero-NN MNIST task! The usage is the same as Homo-NN: we specify our model and loss by nn.CustModel and nn.CustLoss interfaces.

## pipeline initialization

Here we define the pipeline to run a hetero task

In [18]:
import os
import torch as t
from torch import nn
from pipeline import fate_torch_hook
from pipeline.component import HeteroNN
from pipeline.component.hetero_nn import DatasetParam
from pipeline.backend.pipeline import PipeLine
from pipeline.component import Reader, Evaluation, DataTransform
from pipeline.interface import Data, Model
from pipeline.component.nn import save_to_fate

fate_torch_hook(t)

# bind path to fate name&namespace
fate_project_path = os.path.abspath('/data/projects/fate')
guest = 10000
host = 9999

pipeline_img = PipeLine().set_initiator(role='guest', party_id=guest).set_roles(guest=guest, host=host)

guest_data = {"name": "mnist_guest", "namespace": "experiment"}
host_data = {"name": "mnist_host", "namespace": "experiment"}

guest_data_path = fate_project_path + '/examples/data/mnist_csv_guest/labels2.csv'
host_data_path = fate_project_path + '/examples/data/mnist_csv_host/'
pipeline_img.bind_table(name='mnist_guest', namespace='experiment', path=guest_data_path)
pipeline_img.bind_table(name='mnist_host', namespace='experiment', path=host_data_path)

{'namespace': 'experiment', 'table_name': 'mnist_host'}

In [19]:
guest_data = {"name": "mnist_guest", "namespace": "experiment"}
host_data = {"name": "mnist_host", "namespace": "experiment"}
reader_0 = Reader(name="reader_0")
reader_0.get_party_instance(role='guest', party_id=guest).component_param(table=guest_data)
reader_0.get_party_instance(role='host', party_id=host).component_param(table=host_data)

In [20]:
hetero_nn_0 = HeteroNN(name="hetero_nn_0", epochs=2,
                       interactive_layer_lr=0.01, batch_size=512, task_type='classification', seed=100
                       )
guest_nn_0 = hetero_nn_0.get_party_instance(role='guest', party_id=guest)
host_nn_0 = hetero_nn_0.get_party_instance(role='host', party_id=host)

# define model
# use cust model here
# our simple classification model:
"""This should be commented out"""
guest_bottom = t.nn.CustModel(module_name='dummy.py', class_name='DummyNet')
#guest_bottom = None

# use cust model here
host_bottom = t.nn.CustModel(module_name='bottom_net.py', class_name='BottomNet')

# use new top model here
guest_top = t.nn.CustModel(module_name='top_net.py', class_name='TopNet')

# interactive layer define
#interactive_layer = t.nn.InteractiveLayer(out_dim=8, guest_dim=None, host_dim=8)
interactive_layer = t.nn.InteractiveLayer(out_dim=10, guest_dim=1, host_dim=10)

# add models
guest_nn_0.add_top_model(guest_top)
"""This should be commented out"""
guest_nn_0.add_bottom_model(guest_bottom)

host_nn_0.add_bottom_model(host_bottom)

# opt, loss
optimizer = t.optim.Adam(lr=0.01) 
loss = t.nn.CustLoss(loss_module_name='ce', class_name='CrossEntropyLoss')

# use DatasetParam to specify dataset and pass parameters
host_nn_0.add_dataset(DatasetParam(dataset_name='image', return_label=True))
guest_nn_0.add_dataset(DatasetParam(dataset_name='table'))

hetero_nn_0.set_interactive_layer(interactive_layer)
hetero_nn_0.compile(optimizer=optimizer, loss=loss)

In [21]:
pipeline_img.add_component(reader_0)
pipeline_img.add_component(hetero_nn_0, data=Data(train_data=reader_0.output.data))
pipeline_img.add_component(Evaluation(name='eval_0', eval_type='multi'), data=Data(data=hetero_nn_0.output.data))
pipeline_img.compile()

In [22]:
pipeline_img.fit()

2023-10-24 19:30:57.248 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202310241930569912850

2023-10-24 19:30:57.257 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
2023-10-24 19:30:58.280 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:01
2023-10-24 19:30:59.317 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:02
m2023-10-24 19:31:00.334 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-10-24 19:31:00.335 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:03
2023-10-24 19:31:01.351 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:04
2023-10-24 19:31:02.373 | INFO     | pipeline.uti

In [23]:
pipeline_img.get_component('hetero_nn_0').get_output_data()  # get result

,id,label,predict_result,predict_score,predict_detail,type
0,0,7.0,5,0.11435441672801971,"{'0': 0.10275458544492722, '1': 0.085950225591...",train
1,1,2.0,5,0.11435441672801971,"{'0': 0.10275458544492722, '1': 0.085950225591...",train
2,2,1.0,5,0.11435441672801971,"{'0': 0.10275458544492722, '1': 0.085950225591...",train
3,3,0.0,5,0.11435441672801971,"{'0': 0.10275458544492722, '1': 0.085950225591...",train
4,4,4.0,5,0.11435441672801971,"{'0': 0.10275458544492722, '1': 0.085950225591...",train
...,...,...,...,...,...,...
9995,8959,7.0,5,0.11435441672801971,"{'0': 0.10275458544492722, '1': 0.085950225591...",train
9996,9215,2.0,5,0.11435441672801971,"{'0': 0.10275458544492722, '1': 0.085950225591...",train
9997,9471,4.0,5,0.11435441672801971,"{'0': 0.10275458544492722, '1': 0.085950225591...",train
9998,9727,3.0,5,0.11435441672801971,"{'0': 0.10275458544492722, '1': 0.085950225591...",train
